In [ ]:
pip install geopandas

In [ ]:
pip install missingno

In [ ]:
import geopandas as gpd
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Import des bases de données

### TRI (Scénarios Décénal, Centenal, Millénal)

In [ ]:
# import modules
import os

# assign path
path, dirs, files = next(os.walk("/Users/guillaumeallemoniere/Documents/Documents/Jupyter_Notebooks/Cartes_Innondations_VAR/fichiers/"))
# files.remove('.DS_Store')   Rajouter cette commande si le fichier .DS_Store zest présent dans la liste intitulée "files"
file_count = len(files)

dataset_a_supprimer = []

for i in range(file_count):
    if not files[i].endswith(".dbf"):
        dataset_a_supprimer.append(files[i])

for dataset in dataset_a_supprimer:
    files.remove(dataset)


# create empty list
dataframes_list = []

file_count = len(files)

# append datasets to the list
for i in range(file_count):
    temp_df = gpd.read_file("/Users/guillaumeallemoniere/Documents/Documents/Jupyter_Notebooks/Cartes_Innondations_VAR/fichiers/"+files[i])
    dataframes_list.append(temp_df)

In [ ]:
dataset_n_tri_toulon_inondable = []
dataset_n_tri_toulon_iso = []
dataset_n_tri_toulon_enjeu = []
dataset_n_autres = []


for i in range(len(files)):
    if files[i].startswith("n_tri_toulon_inondable") == True:
        dataset_n_tri_toulon_inondable.append(files[i])
    elif files[i].startswith("n_tri_toulon_iso") == True:
        dataset_n_tri_toulon_iso.append(files[i])
    elif files[i].startswith("n_tri_toulon_enjeu") == True:
        dataset_n_tri_toulon_enjeu.append(files[i])
    else:
        dataset_n_autres.append(files[i])

In [ ]:
# display datasets
for (name,dataset) in zip(files,dataframes_list):
    print("Nom du dataset: ", name)
    display(dataset.head(0))
    print("\n\n")

In [ ]:
# display datasets
for (name,dataset) in zip(files,dataframes_list):
    if name in dataset_n_tri_toulon_iso:
        print("Nom du dataset: ", name)
        display(dataset.head(0))
        print("\n\n")

In [ ]:
Data = pd.DataFrame({'uuid':[], 'id':[], 'id_zone':[], 'id_s_inond':[], 'typ_inond':[], 'typ_inond2':[],
       'scenario':[], 'datentree':[], 'datsortie':[], 'ht_min':[], 'ht_max':[], 'est_ref':[],
       'cours_deau':[], 'id_tri':[], 'geometry':[]})

In [ ]:
Data

In [ ]:
for (name,dataset) in zip(files,dataframes_list):
    if (name in dataset_n_tri_toulon_iso): 
        Data = pd.concat([Data,dataset])

In [ ]:
Data.head()

In [ ]:
len(Data)

In [ ]:
msno.bar(Data)

In [ ]:
Data_ISO = Data.drop(columns=['typ_inond2', 'datentree', 'datsortie'])

In [ ]:
msno.bar(Data_ISO)

#### Transform a pandas dataframe to a geopandas dataframe

In [ ]:
# from shapely import wkt
# Data_ISO['geometry']= gpd.GeoSeries.from_wkt(Data_ISO['geometry'])

# Voir le site: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

# Ici, on transforme le pandas dataframes en geopanda data frame

Data_ISO_gdf = gpd.GeoDataFrame(Data_ISO, geometry='geometry')

In [ ]:
print(Data_ISO_gdf.crs)

In [ ]:
Data_ISO_gdf.crs is None

##### Set the correct epsg for the geometry to be correctly mapped later

In [ ]:
# Set epsg to 2154 (RGF93 / Lambert-93, spatial reference for France)

Data_ISO_gdf['geometry'] = Data_ISO_gdf['geometry'].set_crs('epsg:2154')

In [ ]:
Data_ISO_gdf.crs

In [ ]:
Data_ISO_gdf.head()

In [ ]:
# Move epsg to 4326 (WGS84, spatial reference for the world)

Data_ISO_gdf['geometry'] = Data_ISO_gdf['geometry'].to_crs('epsg:4326')

In [ ]:
Data_ISO_gdf = Data_ISO_gdf.reset_index(level=0)

In [ ]:
Data_ISO_gdf.head()

#### Création de 3 dataframe pour les 3 scénarios (Décénal, Centenal, Millénal)

In [ ]:
Data_ISO_gdf['scenario'].unique()

In [ ]:
def scenario(a):
    if (a=='01For'):
        return 1
    elif (a=='02Moy'):
        return 2
    elif (a=='04Fai'):
        return 3
    else:
        return 2

In [ ]:
Data_ISO_gdf['scenario_2'] = list(map(scenario,Data_ISO_gdf['scenario']))

In [ ]:
Data_ISO_gdf.head()

In [ ]:
Data_ISO_gdf['scenario_2'].unique()

In [ ]:
Data_ISO_gdf_Dec = Data_ISO_gdf.loc[Data_ISO_gdf['scenario_2']==1,]
Data_ISO_gdf_Cent = Data_ISO_gdf.loc[Data_ISO_gdf['scenario_2']==2,]
Data_ISO_gdf_Mill = Data_ISO_gdf.loc[Data_ISO_gdf['scenario_2']==3,]

In [ ]:
# Export to csv

Data_ISO_gdf.to_csv('Data.csv')
Data_ISO_gdf_Dec.to_csv('Data_Dec.csv')
Data_ISO_gdf_Cent.to_csv('Data_Cent.csv')
Data_ISO_gdf_Mill.to_csv('Data_Mill.csv')

### PPRN (en complément des TRI importés plus haut)

In [ ]:
zone_pprn = gpd.read_file("/Users/guillaumeallemoniere/Documents/Documents/Jupyter_Notebooks/Cartes_Innondations_VAR/fichier_pprn_83/n_zonages_risque_naturel/zone_pprn_s_083.dbf")

In [ ]:
zone_pprn.head()

#### Import de la base contenant les id GASPAR pour faire le lien avec les PPRN et identifier les types de rsiques (Inondations, Débordement, etc...)

In [ ]:
gaspar = pd.read_excel('GASPAR_PPRN_2.xlsx',sheet_name='Sheet_1')
gaspar = gaspar.rename(columns={"cod_nat_pprn": "id_gaspar"})
gaspar["Compte"]=1

In [ ]:
gaspar.head()

In [ ]:
def est_dans_zone_pprn(a):
    return a in list(zone_pprn["id_gaspar"])

In [ ]:
gaspar["est_dans_pprn"] = list(map(est_dans_zone_pprn,gaspar["id_gaspar"]))

In [ ]:
gaspar.head()

In [ ]:
gaspar = gaspar.loc[(gaspar["est_dans_pprn"] == True),]

In [ ]:
# À présent la base gaspar contient les mêmes dentifiants (id_gaspar) que dans la base des PPRN
# Ce qui sera très utile pour identifier les types de risques de la base des PPRN (grâce à la fonction merge() de pandas)
# et retenir seulement les risques qui nous intéressent.

gaspar

##### Suppression des lignes répétées dans la base

In [ ]:
gaspar = gaspar.groupby(["id_gaspar","lib_risque","est_dans_pprn"]).agg({"Compte":"sum"})

In [ ]:
gaspar

In [ ]:
gaspar = gaspar.reset_index(level=['id_gaspar','lib_risque','est_dans_pprn'])

In [ ]:
gaspar

In [ ]:
gaspar["lib_risque"].unique()

In [ ]:
def last_element(x):
    return ', '.join(x).split(", ")[-1]

In [ ]:
gaspar = gaspar.groupby(["id_gaspar"]).agg({"lib_risque":last_element})
gaspar = gaspar.loc[(gaspar["lib_risque"]!="Mouvement de terrain") & (gaspar["lib_risque"]!="Feu de for√™t"),]
gaspar = gaspar.reset_index(level=['id_gaspar'])
gaspar

In [ ]:
donnees_pprn = pd.merge(zone_pprn, gaspar, on="id_gaspar")

In [ ]:
donnees_pprn.head()

In [ ]:
len(donnees_pprn)

In [ ]:
len(zone_pprn)

In [ ]:
# On voit que la taille de la base des PPRN a diminué suite à nos manipulations avec la base des id_gaspar
# (792 vs 560 lignes)

In [ ]:
zone_pprn = donnees_pprn

In [ ]:
zone_pprn.head()

##### Set the correct epsg for the geometry to be correctly mapped later

In [ ]:
zone_pprn.crs

In [ ]:
# Set epsg to 4326 (WGS84, spatial reference for the world)

zone_pprn['geometry'] = zone_pprn['geometry'].to_crs('epsg:4326')

In [ ]:
msno.bar(zone_pprn)

In [ ]:
zone_pprn.loc[zone_pprn['codezone'].isnull(),]

In [ ]:
zone_pprn = zone_pprn.drop(columns=['urlfic', 'srce_annee'])

In [ ]:
msno.bar(zone_pprn)

In [ ]:
zone_pprn = zone_pprn.dropna(subset=['nom', 'codezone'])

In [ ]:
msno.bar(zone_pprn)

In [ ]:
zone_pprn.head()

In [ ]:
zone_pprn["codezone"].unique()

#### Création de 10 dataframes pour les 10 codes

In [ ]:
zone_pprn_Zone_rouge = zone_pprn.loc[zone_pprn['codezone']=='Zone rouge',]
zone_pprn_B1 = zone_pprn.loc[zone_pprn['codezone']=='B1',]
zone_pprn_B2 = zone_pprn.loc[zone_pprn['codezone']=='B2',]
zone_pprn_ZBH = zone_pprn.loc[zone_pprn['codezone']=='ZBH',]
zone_pprn_ZH = zone_pprn.loc[zone_pprn['codezone']=='ZH',]
zone_pprn_NR = zone_pprn.loc[zone_pprn['codezone']=='NR',]
zone_pprn_Alea_exceptionnel = zone_pprn.loc[zone_pprn['codezone']=='Alea exceptionnel',]
zone_pprn_B3 = zone_pprn.loc[zone_pprn['codezone']=='B3',]
zone_pprn_R1 = zone_pprn.loc[zone_pprn['codezone']=='R1',]
zone_pprn_R2 = zone_pprn.loc[zone_pprn['codezone']=='R2',]

In [ ]:
zone_pprn["lib_risque"].unique()

### Kepler Gl

###### ATTENTION ! pour la partie portant sur la visualisation avec kepler.gl il est nécessaire d'installer certains packages notamment certains widgets pour pouvoir afficher la carte correctement.

###### Les liens utilisés pour l'installation de ces packages sont les suivants:
https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f (voir la partie installation tout en bas de la page)

https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html


https://stackoverflow.com/questions/57352649/installing-jupyter-extension-failure-to-enable-nbextension

In [ ]:
# Voir le lien: 
# https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f

# Si la commande "pip install" ne fonctionne pas penser à écrire "!pip install" pour installer le package voulu.
# Voir le lien: https://stackoverflow.com/questions/45767252/cant-use-pip-in-jupyter-notebook

!pip install ipywidgets

In [ ]:
# Voir le lien
# https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f

# Si la commande "pip install" ne fonctionne pas penser à écrire "!pip install" pour installer le package voulu.
# Voir le lien: https://stackoverflow.com/questions/45767252/cant-use-pip-in-jupyter-notebook

!pip install keplergl

In [ ]:
# Pour voir la liste des packages installés: !pip list

# Pour supprimer un package installé: !pip uninstall -y keplergl

In [ ]:
# Voir le lien
# https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html

# Si la commande "pip install" ne fonctionne pas penser à écrire "!pip install" pour installer le package voulu.
# Voir le lien: https://stackoverflow.com/questions/45767252/cant-use-pip-in-jupyter-notebook

!pip install jupyter_contrib_nbextensions

In [ ]:
# Voir les liens
# https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
# https://stackoverflow.com/questions/57352649/installing-jupyter-extension-failure-to-enable-nbextension

!jupyter contrib nbextension install --user

In [ ]:
# Voir les liens
# https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
# https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f

!jupyter contrib nbextension install --py --sys-prefix keplergl

In [ ]:
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1

In [ ]:
type(Data_ISO)

In [ ]:
# from shapely import wkt

# Data_ISO['geometry']= gpd.GeoSeries.from_wkt(Data_ISO['geometry'])

# Voir le site: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

Data_ISO_gdf = gpd.GeoDataFrame(Data_ISO, geometry='geometry')

In [ ]:
type(Data_ISO_gdf)

In [ ]:
type(Data_ISO_gdf['geometry'])

In [ ]:
map_1.add_data(data=Data_ISO_gdf, name='Data_ISO')

In [ ]:
map_1.save_to_html(file_name='Map_BAS_RHIN.html')

In [ ]:
Data_ISO_gdf.to_csv("Data_TRI_BAS_RHIN.csv")

### Folium

In [ ]:
# Exporting the data frame Data_ISO_gdf and others to a json file in local folders

Data_ISO_gdf.to_file('Data_ISO_gdf.json', driver="GeoJSON")

# TRI Databases
Data_ISO_gdf_Dec.to_file('Data_ISO_gdf_Dec.json', driver="GeoJSON")
Data_ISO_gdf_Cent.to_file('Data_ISO_gdf_Cent.json', driver="GeoJSON")
Data_ISO_gdf_Mill.to_file('Data_ISO_gdf_Mill.json', driver="GeoJSON")

# PPRN Databases
zone_pprn_Zone_rouge.to_file('zone_pprn_Zone_rouge.json', driver="GeoJSON")
zone_pprn_B1.to_file('zone_pprn_B1.json', driver="GeoJSON")
zone_pprn_B2.to_file('zone_pprn_B2.json', driver="GeoJSON")
zone_pprn_ZBH.to_file('zone_pprn_ZBH.json', driver="GeoJSON")
zone_pprn_ZH.to_file('zone_pprn_ZH.json', driver="GeoJSON")
zone_pprn_NR.to_file('zone_pprn_NR.json', driver="GeoJSON")
zone_pprn_Alea_exceptionnel.to_file('zone_pprn_Alea_exceptionnel.json', driver="GeoJSON")
zone_pprn_B3.to_file('zone_pprn_B3.json', driver="GeoJSON")
zone_pprn_R1.to_file('zone_pprn_R1.json', driver="GeoJSON")
zone_pprn_R2.to_file('zone_pprn_R2.json', driver="GeoJSON")

In [ ]:
# Loading the json file that has been exported

import json

# TRI Databases
Data = json.load(open('Data_ISO_gdf.json','r'))
Data_Dec = json.load(open('Data_ISO_gdf_Dec.json','r'))
Data_Cent = json.load(open('Data_ISO_gdf_Cent.json','r'))
Data_Mill = json.load(open('Data_ISO_gdf_Mill.json','r'))


# PPRN Databases
Data_Zone_rouge = json.load(open('zone_pprn_Zone_rouge.json','r'))
Data_B2 = json.load(open('zone_pprn_B2.json','r'))
Data_ZBH = json.load(open('zone_pprn_ZBH.json','r'))
Data_Zone_1 = json.load(open('zone_pprn_Zone_1.json','r'))
Data_B1 = json.load(open('zone_pprn_B1.json','r'))
Data_R1 = json.load(open('zone_pprn_R1.json','r'))
Data_R2 = json.load(open('zone_pprn_R2.json','r'))
Data_Zone_2 = json.load(open('zone_pprn_Zone_2.json','r'))
Data_Alea_exceptionnel = json.load(open('zone_pprn_Alea_exceptionnel.json','r'))
Data_B3 = json.load(open('zone_pprn_B3.json','r'))
Data_ZH = json.load(open('zone_pprn_ZH.json','r'))
Data_NR = json.load(open('zone_pprn_NR.json','r'))

In [ ]:
# Creating new keys 'uuid' and 'scenario' in the json file
# Only for the files Data, Data_Dec, Data_Cent, Data_Mill.


for feature in Data['features']:
    feature['uuid'] = feature['properties']['uuid']  
    feature['scenario'] = feature['properties']['scenario_2']

    

for feature_Dec in Data_Dec['features']:
    feature_Dec['uuid'] = feature_Dec['properties']['uuid']  
    feature_Dec['scenario'] = feature_Dec['properties']['scenario_2']
    
for feature_Cent in Data_Cent['features']:
    feature_Cent['uuid'] = feature_Cent['properties']['uuid']  
    feature_Cent['scenario'] = feature_Cent['properties']['scenario_2']

for feature_Mill in Data_Mill['features']:
    feature_Mill['uuid'] = feature_Mill['properties']['uuid']  
    feature_Mill['scenario'] = feature_Mill['properties']['scenario_2']

In [ ]:
Data_Dec['features'][0].keys()

In [ ]:
Data_Cent['features'][0].keys()

In [ ]:
Data_Mill['features'][0].keys()

#### Choropleth (TRI: Scénarios Décénal, Centenal et Millénal)

In [ ]:
# creating a folium map centered around the city of Strasbourg

import folium

m = folium.Map(location=[43.124228, 5.928], tiles="Stamen Terrain", zoom_start=12)
m

In [ ]:
# Adding colored zones depending on the flood risk level

c = folium.Choropleth(
        geo_data=Data,
        data=Data_ISO_gdf,
        columns=["uuid", "scenario_2"],
        key_on="feature.uuid",
        fill_color="",
        fill_opacity=0.8,
        line_opacity=1,
        name="Décénale",
    )

# Remove the legend at the top right corner
# https://www.roelpeters.be/remove-the-legend-in-a-folium-choropleth-map/

for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])

        
# and finally adding a tooltip/hover to the choropleth's geojson
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid']).add_to(c.geojson)
    
    
c.add_to(m)



folium.LayerControl().add_to(m)

# Pour choisir les codes couleurs penser à taper "ColorBrewer Code" comme mot clé sur Google.
#folium.LayerControl().add_to(m)

In [ ]:
# c.geojson.data['features']

# c.geojson

In [ ]:
# Voir le site https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe
# pour des explications sur cette partie avec du code HTML

# We import the required library:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Périodes de retour</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#06519A;opacity:1;'></span>1000 ans.</li>
    <li><span style='background:#6AAED4;opacity:1;'></span>100 ans.</li>
    <li><span style='background:#EFF3FF;opacity:1;'></span>10 ans.</li>
    

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m

In [ ]:
# Exporting the map in HTML format

m.save("Carte_Inondations_Toulon_Choropleth.html")

In [ ]:
# iso_ht_01_Moyen

#### GeoJson (TRI: Scénarios Décénal, Centenal et Millénal + PPRN)

In [ ]:
import folium

m = folium.Map(location=[43.124228, 5.928], tiles="Stamen Terrain", zoom_start=12)



c_10_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#EFF3FF',
    'fillOpacity': 0.7
}

c_10 = folium.GeoJson(
    data=(open("Data_ISO_gdf_Dec.json", 'r').read()),
    name="Décénale",
    style_function=lambda x: c_10_bordersStyle)

c_10.add_to(m)



c_100_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#6AAED4',
    'fillOpacity': 0.7
}

c_100 = folium.GeoJson(
    data=(open("Data_ISO_gdf_Cent.json", 'r').read()),
    name="Centenale",
    style_function=lambda x: c_100_bordersStyle)

c_100.add_to(m)



c_1000_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#06519A',
    'fillOpacity': 0.7
}

c_1000 = folium.GeoJson(
    data=(open("Data_ISO_gdf_Mill.json", 'r').read()),
    name="Millénale",
    style_function=lambda x: c_1000_bordersStyle)

c_1000.add_to(m)



c_pprn_Zone_rouge_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#BB0B0B',
    'fillOpacity': 0.7
}

c_pprn_Zone_rouge = folium.GeoJson(
    data=Data_Zone_rouge,
    name="PPRN ZONE ROUGE",
    style_function=lambda x: c_pprn_Zone_rouge_bordersStyle)

c_pprn_Zone_rouge.add_to(m)



c_pprn_B2_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#008000',
    'fillOpacity': 0.7
}

c_pprn_B2 = folium.GeoJson(
    data=Data_B2,
    name="PPRN B2",
    style_function=lambda x: c_pprn_B2_bordersStyle)

c_pprn_B2.add_to(m)



c_pprn_ZBH_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#5DADE2',
    'fillOpacity': 0.7
}

c_pprn_ZBH = folium.GeoJson(
    data=Data_ZBH,
    name="PPRN ZBH",
    style_function=lambda x: c_pprn_ZBH_bordersStyle)

c_pprn_ZBH.add_to(m)





c_pprn_B1_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#2874A6',
    'fillOpacity': 0.7
}

c_pprn_B1 = folium.GeoJson(
    data=Data_B1,
    name="PPRN B1",
    style_function=lambda x: c_pprn_B1_bordersStyle)

c_pprn_B1.add_to(m)



c_pprn_R1_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#922B21',
    'fillOpacity': 0.7
}

c_pprn_R1 = folium.GeoJson(
    data=Data_R1,
    name="PPRN R1",
    style_function=lambda x: c_pprn_R1_bordersStyle)

c_pprn_R1.add_to(m)



c_pprn_R2_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#EC7063',
    'fillOpacity': 0.7
}

c_pprn_R2 = folium.GeoJson(
    data=Data_R2,
    name="PPRN R2",
    style_function=lambda x: c_pprn_R2_bordersStyle)

c_pprn_R2.add_to(m)




c_pprn_Alea_exceptionnel_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#2ECC71',
    'fillOpacity': 0.7
}

c_pprn_Alea_exceptionnel = folium.GeoJson(
    data=Data_Alea_exceptionnel,
    name="PPRN Alea Exceptionnel",
    style_function=lambda x: c_pprn_Alea_exceptionnel_bordersStyle)

c_pprn_Alea_exceptionnel.add_to(m)



c_pprn_B3_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#AED6F1',
    'fillOpacity': 0.7
}

c_pprn_B3 = folium.GeoJson(
    data=Data_B3,
    name="PPRN B3",
    style_function=lambda x: c_pprn_B3_bordersStyle)

c_pprn_B3.add_to(m)



c_pprn_ZH_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#F8C471',
    'fillOpacity': 0.7
}

c_pprn_ZH = folium.GeoJson(
    data=Data_ZH,
    name="PPRN ZH",
    style_function=lambda x: c_pprn_ZH_bordersStyle)

c_pprn_ZH.add_to(m)



c_pprn_NR_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#13EFF3',
    'fillOpacity': 0.7
}

c_pprn_NR = folium.GeoJson(
    data=Data_NR,
    name="PPRN NR",
    style_function=lambda x: c_pprn_NR_bordersStyle)

c_pprn_NR.add_to(m)





# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_10)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_100)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_1000)



# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_Zone_rouge)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_B2)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_ZBH)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_B1)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_R1)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_R2)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_Alea_exceptionnel)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_B3)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_ZH)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['codezone','lib_risque','id_zone']).add_to(c_pprn_NR)


folium.LayerControl().add_to(m)

In [ ]:
# https://www.youtube.com/watch?v=h16O4xt6yBU

# https://nagasudhir.blogspot.com/2021/07/draw-borders-from-geojson-paths-in.html

In [ ]:
# Interactive map with layers selection:

# https://kodu.ut.ee/~kmoch/geopython2019/L6/interactive-map-folium.html

In [ ]:
# https://stackoverflow.com/questions/55088688/how-do-you-add-geojsontooltip-to-folium-choropleth-class-in-folium

In [ ]:
# tooltip=folium.features.Tooltip(fields=['feature.properties.county_name'])

In [ ]:
# Voir le site https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe
# pour des explications sur cette partie avec du code HTML

# We import the required library:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Périodes de retour (TRI)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#06519A;opacity:1;'></span>1000 ans.</li>
    <li><span style='background:#6AAED4;opacity:1;'></span>100 ans.</li>
    <li><span style='background:#EFF3FF;opacity:1;'></span>10 ans.</li>
  </ul>
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>

{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m

In [ ]:
# Exporting the map in HTML format

m.save("Carte_Inondations_Toulon_Geojson.html")

In [ ]:
# https://www.youtube.com/watch?v=h16O4xt6yBU

# https://nagasudhir.blogspot.com/2021/07/draw-borders-from-geojson-paths-in.html

In [ ]:
# Interactive map with layers selection:

# https://kodu.ut.ee/~kmoch/geopython2019/L6/interactive-map-folium.html

In [ ]:
# https://stackoverflow.com/questions/55088688/how-do-you-add-geojsontooltip-to-folium-choropleth-class-in-folium

In [ ]:
# tooltip=folium.features.Tooltip(fields=['feature.properties.county_name'])

In [ ]:
# https://automating-gis-processes.github.io/CSC18/lessons/L4/point-in-polygon.html#reading-kml-files-in-geopandas

In [ ]:
def type_a(a):
    return str(type(a))

In [ ]:
Data_ISO_gdf_Cent["Geometry Type"] = list(map(type_a,Data_ISO_gdf_Cent["geometry"]))

In [ ]:
Data_ISO_gdf_Cent.head()

In [ ]:
Data_ISO_gdf_Cent["Geometry Type"].unique()

In [ ]:
Data_ISO_gdf_Cent.loc[Data_ISO_gdf_Cent["Geometry Type"]=="<class 'shapely.geometry.multipolygon.MultiPolygon'>",]

In [ ]:
for i in Data_ISO_gdf_Cent.loc[[3253],"geometry"]:
    print(i)

In [ ]:
list(Data_ISO_gdf_Cent.loc[[3253],"geometry"].geoms)

In [ ]:
type(Data_ISO_gdf_Cent.loc[[3253],"geometry"])

In [ ]:
from shapely import Polygon
ob = gpd.geoseries(MultiPolygon([
    (
    ((0.0, 0.0), (0.0, 1.0), (1.0, 1.0), (1.0, 0.0)),
    [((0.1,0.1), (0.1,0.2), (0.2,0.2), (0.2,0.1))]
    )
]))

In [ ]:
type(ob)

In [ ]:
from shapely import Polygon, MultiPolygon
ob = MultiPolygon(list(Data_ISO_gdf_Cent.loc[[3253],"geometry"]))
len(ob.geoms)
1
type(ob.geoms[0]) == Polygon

In [ ]:
Data_ISO_gdf_Cent.loc[[3253,3256,6518],].explode()

In [ ]:
type(Data_ISO_gdf_Cent.loc[[3256],"geometry"].geometry.iloc[0])

In [ ]:
liste = [i for i in Data_ISO_gdf_Cent.loc[[3253],"geometry"]]
df.geometry.iloc[0]

In [ ]:
liste

In [ ]:
def extract_coordinates(Polygon):
    if Polygon.geom_type == 'Polygon':
        x,y = Polygon.exterior.coords.xy
        # return np.dstack(np.array([x.tolist(),y.tolist()]))
        return np.array([x.tolist(),y.tolist()]).transpose()
    else:
        return list(map(extract_coordinates,list(Polygon)))

In [ ]:
def extract_multipolygon(liste):
    liste2 = []
    for i in liste:
        liste2.append(i)
    return liste2

In [ ]:
Data_ISO_gdf_Cent.explode()

In [ ]:
Data_ISO_gdf_Cent.loc[[3253],]

In [ ]:
Data_ISO_gdf_Cent.loc[type(Data_ISO_gdf_Cent["geometry"])==]

In [ ]:
Data_ISO_gdf_Cent["geometry"]

In [ ]:
# https://stackoverflow.com/questions/64099107/convert-multipolygon-geometry-into-list

In [ ]:
def type_a(a):
    return type(a)

In [ ]:
list(map(type_a,Data_ISO_gdf_Cent["geometry"]))

In [ ]:
import pandas as pd
Data = pd.DataFrame({"type":list(map(type_a,Data_ISO_gdf_Cent["geometry"]))})

In [ ]:
Data.head()

In [ ]:
Data["type"].unique()

In [ ]:
Data_ISO_gdf_Cent["geometry"].unique()

In [ ]:
# https://gis.stackexchange.com/questions/287306/listing-all-polygon-vertices-coordinates-using-geopandas

import numpy as np
x,y = Data_ISO_gdf_Cent["geometry"][0].exterior.coords.xy
# np.dstack(np.array([x.tolist(),y.tolist()]))
np.array([x.tolist(),y.tolist()]).transpose()

In [ ]:
np.array([x.tolist(),y.tolist()])

In [ ]:
np.array([x.tolist(),y.tolist()]).transpose()